Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Yajie Qi,427150"
COLLABORATORS = "Danqi Zhao,441469","Haoyu Tang,430191"

---

# Shape Structures
In this notebook we will implement a region growing algorithm for shape segmentation and a function for cylinder fitting

In [2]:
import numpy as np
import openmesh as om
from scipy.linalg import eigh
from queue import Queue
import k3d

In [3]:
mesh = om.read_trimesh("spot/spot_triangulated.obj")
feature = np.load("features.npy")

## Region Growing

In this task you will implement a function `region_growing(mesh, feature, seed_thres, merge_thres)`
As inputs this function will get a mesh, a feature vector with a floating value for each face (which we already precomputed for you), as well as two thresholds to start and stop the growing.

every face with a feature value above `seed_thres` is a potential seed for the growing algorithm. The corresponding region should grow as long as there is a neighbouring face with a feature value above `merge_thres`.\
To implement this task a simple Queue is enough, as the order in which you process the face does not matter. For this you can use the `Queue` class.\
The function should output a list of lists, where you save the face indices of the respective clusters.

In [4]:
def region_growing(mesh, feature, seed_thres, merge_thres):
    ### BEGIN SOLUTION

    queue = Queue()
    clusters = []
    clustered = []

    seeds = []
    for fh in mesh.faces():
        if feature[fh.idx()] > seed_thres:
            seeds.append(fh)

    while seeds:
        seed = seeds[0]

        queue = Queue()

        queue.put(seed)

        cluster = []

        while not queue.empty():

            s = queue.get()

            if (s.idx() not in clustered and feature[s.idx()] > merge_thres):
                cluster.append(s.idx())
                clustered.append(s.idx())

                if s in seeds:
                    seeds.remove(s)

                for fh in mesh.ff(s):
                    if (fh.idx() not in clustered):
                        queue.put(fh)

        clusters.append(cluster)

    return clusters
    

    ### END SOLUTION

def cluster_to_labels(cluster_list):
    faces = np.empty((0,),dtype='long')
    test_feature = np.zeros(feature.shape)
    for i, cluster in enumerate(cluster_list):    
        for f in cluster:
            for v in mesh.fv(mesh.face_handle(f)):
                test_feature[v.idx()] = i + 1
    return test_feature

In [5]:
cluster_list = region_growing(mesh, feature, 8.5, 7.5)

In [6]:
vis = cluster_to_labels(cluster_list)
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=vis, 
                 color_map=k3d.colormaps.matplotlib_color_maps.viridis)
plot

E:\Conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
E:\Conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [7]:
mesh = om.read_trimesh("spot/spot_triangulated.obj")
feature = np.load("features.npy")
cluster_list = region_growing(mesh, feature, 8.5, 7.5)
assert(len(cluster_list) == 9)
assert(sum(len(cluster_list[i]) for i in range(9)) == 1118)


# Cylinder estimation

In the next task you will estimate the parameters of a given cylinder. This consists of two separate tasks:
- You need to build the discrete shape operator in the function `shape_operator(mesh)`
- You solve the circle (2D sphere) fitting problem in the function `circle_estimation(points)`

We provide several helper functions for you to solve these problems.\
To help you with constructing the shape operator you can compute the dihedral angle with `mesh.calc_dihedral_angle(eh)`. You should however skip boundary edges, as for these the dihedral angle is not defined. You can check for boundary edges with `mesh.is_boundary(eh)`.

To fit a circle you need to solve a linear least squares system, for which you can use the function `np.linalg.lstsq(A,b)`.

You only need to deal with these two functions as we provide a function `cylinder_estimation(mesh)`, that calls them with the appropriate arguments. This function will compute the eigenvectors of the shape operator, transform the cylinder accordingly and call `circle_estimation(points)` with the transformed points.

In [8]:
def rotation_matrix_from_vectors(vec1, vec2):
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

def transform_mesh(mesh, radius, direction, center):
    direction = direction / np.linalg.norm(direction)
    center = center - np.dot(center,direction) * direction
    default_radius = np.linalg.norm(mesh.points()[:,:2],axis=1).mean()
    rot = rotation_matrix_from_vectors(np.array([0,0,1]), direction)
    mesh.points()[:] = mesh.points() @ rot.transpose(1,0)
    mesh.points()[:] *= radius / default_radius
    mesh.points()[:] += center
    return direction, center, radius
    
def shape_operator(mesh):
    ### BEGIN SOLUTION

    coords = mesh.points()

    cov_mat = np.zeros((3,3))
    for eh in mesh.edges():

        if (not mesh.is_boundary(eh)):
        
            v1 = mesh.to_vertex_handle(mesh.halfedge_handle(eh, 0)).idx()
            v2 = mesh.to_vertex_handle(mesh.halfedge_handle(eh, 1)).idx()
                    
            v = np.matrix(coords[v1] - coords[v2])
            edge_norm = np.linalg.norm(v)
            
            dihedral_angle = mesh.calc_dihedral_angle(eh)
            unit_v = v / edge_norm

            cov_mat += edge_norm * dihedral_angle * np.dot(unit_v.T, unit_v)
        
    return cov_mat

    ### END SOLUTION
        
def circle_estimation(points):
    ### BEGIN SOLUTION

    n = len(points)
    A = np.zeros((n, 3))
    b = np.zeros((n, 1))
    
    for i, p in enumerate(points):
        px = p[0]
        py = p[1]
        A[i] = [2.0 * px, 2.0 * py, 1.0]
        b[i] = px * px + py * py
        
    w = np.linalg.lstsq(A, b, rcond=-1)[0]
    
    c_x = w[0][0]
    c_y = w[1][0]
    r = np.sqrt(w[2][0] + c_x * c_x + c_y * c_y)
    
    return c_x, c_y, r


    ### END SOLUTION

def cylinder_estimation(mesh):
    so = shape_operator(mesh)
    (eigvalues, eigvectors) = eigh(so)
    rot = rotation_matrix_from_vectors(eigvectors[:,2], np.array([0,0,1]))
    points = mesh.points() @ rot.transpose(1,0)
    c_x, c_y, r = circle_estimation(points)
    c = np.array([c_x,c_y,0]) @ rot
    return eigvectors[:,2], c, r

In [9]:
mesh = om.read_trimesh("cylinder.off")
predictions = transform_mesh(mesh, 2.0, np.array([1,1,1]), np.array([-1,1,1]))
estimate = cylinder_estimation(mesh)
print(predictions)
print(estimate)

(array([0.57735027, 0.57735027, 0.57735027]), array([-1.33333333,  0.66666667,  0.66666667]), 2.0)
(array([-0.57735006, -0.57735045, -0.5773503 ]), array([-1.33333294,  0.66666581,  0.66666639]), 2.0000032778349213)


In [10]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices())
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [11]:
mesh = om.read_trimesh("cylinder.off")
predictions = transform_mesh(mesh, 0.5, np.array([1,0,1]), np.array([0,1,0]))
estimate = cylinder_estimation(mesh)
assert(min(np.abs(predictions[0] - estimate[0]).mean(),np.abs(predictions[0] + estimate[0]).mean()) < 1e-5)

In [12]:
mesh = om.read_trimesh("cylinder.off")
predictions = transform_mesh(mesh, 3.0, np.array([0,1,1]), np.array([1,1,1]))
estimate = cylinder_estimation(mesh)
assert(np.abs(predictions[1] - estimate[1]).mean() < 1e-5)
assert(np.abs(predictions[2] - estimate[2]) < 1e-5)